In [1]:
import mysql.connector
from pyspark.sql import SparkSession
import csv
import random
from pyspark.sql.functions import lit, col, when

In [2]:
class Request_SQL:
    
    DFs = {}
    
    def __init__(self, spark = "temp_spark", files = None, ):
        self.spark = SparkSession.builder.appName(spark).getOrCreate()
        self.read_files(files)
        
        
    def select(self, table, columns='*', where="", join=""):
        
        if join != "":
            join = f"JOIN {join} on {table}.{join} = {join}.id"
            print(join)
        if where != "":
            where = f"WHERE {where}"
            print(where)    
        
        query = f'''
                SELECT {columns}
                FROM {table} 
                {join}
                {where}
                '''
        print(query)
        # Exécution d'une requête SQL pour sélectionner les héros avec plus de 100 points de vie
        result = self.spark.sql(query)

        # Affichage des résultats
        result.show(n=500, truncate=False)
    
        


    def update(self, table, column, conditon):
                
        updated_df = self.DFs[table].withColumn(f'{column}_updated', condition.otherwise(col(column)))
        # Mettre à jour la colonne "armor" avec les valeurs de la colonne "armor_updated"
        updated_df = updated_df.withColumn(column, col(f'{column}_updated'))
        # Supprimer la colonne "armor_updated"
        updated_df = updated_df.drop(f'{column}_updated')
        # updated_df.show()
        self.DFs[table] = updated_df
        self.DFs[table].createOrReplaceTempView(table)

    
    def set_DFs(self, file_name):
        # print(self.get_name_file(file_name))
        if "csv" in file_name.split('.')[-1:]:
            self.DFs[self.get_name_file(file_name)] = self.spark.read.csv(f"assets/csv/{self.get_name_file(file_name)}.csv", header=True, inferSchema=True)
            self.DFs[self.get_name_file(file_name)].createOrReplaceTempView(self.get_name_file(file_name))
            
    
    def read_files(self, files):         
        if files is not None:            
            if isinstance(files, list):
                for key, val in enumerate(files):                 
                    self.set_DFs(val)
            else:                            
                self.set_DFs(files)                   
    
    
    #############################
    ########### UTILS ###########
    #############################
    def get_name_file(self, file):
        return file.split('.')[-2:-1:][0].split('/')[-1:][0]
    
    def print_DF(self, name):
        print(self.DFs[name], "\n")
        
    
    def show_DF(self, name, n=50):
        self.DFs[name].show(n=n)
        
    def show_DFs(self,n=50):
        for table in self.DFs:
            self.DFs[table].show(n=n)
                    
    def print_DFs(self):
        for key,  df in self.DFs.items():
            print(df, "\n")
    
    def print_names_DFs(self):
        for df in self.DFs:
            print(df, "\n")
            
    def print_table(self, name):
        print(self.spark.table(name), "\n")
                    
    def print_tables(self):
        for table in self.spark.catalog.listTables():
            print(table, "\n")
            

                    
    

In [3]:
request = Request_SQL(spark = "temp_hero_test", files = ["hero.csv", "armor.csv", "weapon.csv"])

In [4]:
request.print_DFs()
request.print_DF("hero")
request.print_tables()
request.print_table("hero")
request.print_names_DFs()

DataFrame[id: int, name: string, pv: int, max_pv: int, atk: int, atk_magik: int, mana: int, max_mana: int, initiative: int, experience: int, experience_to_level_up: int, level: int, speed: int, origine_speed: int, items: string, weapon: int, armor: string] 

DataFrame[id: int, name: string, armor: int, armor_magik: int, mana: int, durability: int, max_durability: int, broken: boolean] 

DataFrame[id: int, name: string, atk: int, atk_magik: int, mana: int, mana_cost: int, durability: int, max_durability: int, broken: boolean] 

DataFrame[id: int, name: string, pv: int, max_pv: int, atk: int, atk_magik: int, mana: int, max_mana: int, initiative: int, experience: int, experience_to_level_up: int, level: int, speed: int, origine_speed: int, items: string, weapon: int, armor: string] 

Table(name='armor', catalog=None, namespace=[], description=None, tableType='TEMPORARY', isTemporary=True) 

Table(name='hero', catalog=None, namespace=[], description=None, tableType='TEMPORARY', isTemporary

In [5]:
request.print_names_DFs()

hero 

armor 

weapon 



In [6]:
request.select("hero", "*", "hero.weapon is not null", "weapon")

JOIN weapon on hero.weapon = weapon.id
WHERE hero.weapon is not null

                SELECT *
                FROM hero 
                JOIN weapon on hero.weapon = weapon.id
                WHERE hero.weapon is not null
                
+---+------+---+------+---+---------+----+--------+----------+----------+----------------------+-----+-----+-------------+-----+------+-----+---+---------+---+---------+----+---------+----------+--------------+------+
|id |name  |pv |max_pv|atk|atk_magik|mana|max_mana|initiative|experience|experience_to_level_up|level|speed|origine_speed|items|weapon|armor|id |name     |atk|atk_magik|mana|mana_cost|durability|max_durability|broken|
+---+------+---+------+---+---------+----+--------+----------+----------+----------------------+-----+-----+-------------+-----+------+-----+---+---------+---+---------+----+---------+----------+--------------+------+
|26 |Zander|157|157   |27 |38       |74  |74      |70        |366       |437                   |3    |82  

In [7]:
condition = when(col('id') == 18, 18)
request.update("hero", "armor", condition)

In [8]:
request.select("hero", "*", "hero.armor is not null", "armor")

JOIN armor on hero.armor = armor.id
WHERE hero.armor is not null

                SELECT *
                FROM hero 
                JOIN armor on hero.armor = armor.id
                WHERE hero.armor is not null
                
+---+-------+---+------+---+---------+----+--------+----------+----------+----------------------+-----+-----+-------------+-----+------+-----+---+---------------+-----+-----------+----+----------+--------------+------+
|id |name   |pv |max_pv|atk|atk_magik|mana|max_mana|initiative|experience|experience_to_level_up|level|speed|origine_speed|items|weapon|armor|id |name           |armor|armor_magik|mana|durability|max_durability|broken|
+---+-------+---+------+---+---------+----+--------+----------+----------+----------------------+-----+-----+-------------+-----+------+-----+---+---------------+-----+-----------+----+----------+--------------+------+
|18 |Raphael|171|171   |35 |4        |96  |96      |38        |903       |860                   |4    |60   |27

In [9]:
request.show_DF('hero')

+---+---------+---+------+---+---------+----+--------+----------+----------+----------------------+-----+-----+-------------+-----+------+-----+
| id|     name| pv|max_pv|atk|atk_magik|mana|max_mana|initiative|experience|experience_to_level_up|level|speed|origine_speed|items|weapon|armor|
+---+---------+---+------+---+---------+----+--------+----------+----------+----------------------+-----+-----+-------------+-----+------+-----+
|  1|     Aria|189|   189|  1|       23|  35|      35|        61|        81|                   901|    5|   99|           11|   []|  null| null|
|  2|Balthazar|162|   162| 29|       49|  67|      67|        32|       613|                   422|    5|   88|           23|   []|  null| null|
|  3|Cassandra|103|   103| 43|       39|  50|      50|        88|       817|                   788|    8|   90|           87|   []|  null| null|
|  4|   Darian| 75|    75| 30|       14|  66|      66|        86|       101|                   112|    6|   81|           60|   []

In [10]:
request.show_DFs()

+---+---------+---+------+---+---------+----+--------+----------+----------+----------------------+-----+-----+-------------+-----+------+-----+
| id|     name| pv|max_pv|atk|atk_magik|mana|max_mana|initiative|experience|experience_to_level_up|level|speed|origine_speed|items|weapon|armor|
+---+---------+---+------+---+---------+----+--------+----------+----------+----------------------+-----+-----+-------------+-----+------+-----+
|  1|     Aria|189|   189|  1|       23|  35|      35|        61|        81|                   901|    5|   99|           11|   []|  null| null|
|  2|Balthazar|162|   162| 29|       49|  67|      67|        32|       613|                   422|    5|   88|           23|   []|  null| null|
|  3|Cassandra|103|   103| 43|       39|  50|      50|        88|       817|                   788|    8|   90|           87|   []|  null| null|
|  4|   Darian| 75|    75| 30|       14|  66|      66|        86|       101|                   112|    6|   81|           60|   []

In [11]:
condition = when(col('id').between(10, 15), 11)
request.update("hero", "armor", condition)

In [12]:
request.select("hero", "*", "hero.armor is not null", "armor")

JOIN armor on hero.armor = armor.id
WHERE hero.armor is not null

                SELECT *
                FROM hero 
                JOIN armor on hero.armor = armor.id
                WHERE hero.armor is not null
                
+---+-------+---+------+---+---------+----+--------+----------+----------+----------------------+-----+-----+-------------+-----+------+-----+---+-----------------+-----+-----------+----+----------+--------------+------+
|id |name   |pv |max_pv|atk|atk_magik|mana|max_mana|initiative|experience|experience_to_level_up|level|speed|origine_speed|items|weapon|armor|id |name             |armor|armor_magik|mana|durability|max_durability|broken|
+---+-------+---+------+---+---------+----+--------+----------+----------+----------------------+-----+-----+-------------+-----+------+-----+---+-----------------+-----+-----------+----+----------+--------------+------+
|10 |Jasper |183|183   |22 |1        |29  |29      |58        |962       |154                   |2    |55

In [13]:

condition = when((col('armor').isNull()) & (col('id') < 5), 7)
request.update("hero", "armor", condition)
request.select("hero", "*", "hero.armor is not null", "armor")

JOIN armor on hero.armor = armor.id
WHERE hero.armor is not null

                SELECT *
                FROM hero 
                JOIN armor on hero.armor = armor.id
                WHERE hero.armor is not null
                
+---+---------+---+------+---+---------+----+--------+----------+----------+----------------------+-----+-----+-------------+-----+------+-----+---+-----------------+-----+-----------+----+----------+--------------+------+
|id |name     |pv |max_pv|atk|atk_magik|mana|max_mana|initiative|experience|experience_to_level_up|level|speed|origine_speed|items|weapon|armor|id |name             |armor|armor_magik|mana|durability|max_durability|broken|
+---+---------+---+------+---+---------+----+--------+----------+----------+----------------------+-----+-----+-------------+-----+------+-----+---+-----------------+-----+-----------+----+----------+--------------+------+
|1  |Aria     |189|189   |1  |23       |35  |35      |61        |81        |901                   |